<a href="https://colab.research.google.com/github/mayank1365/Youtube_comment_scrapper/blob/main/Youtube_comment_scrapper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install selenium
!pip install webdriver_manager
!pip install pandas
!pip install transformers
!pip install youtube-comment-downloader
!pip install pytube
!pip install youtube-comment-downloader faiss-cpu
!pip install sentence-transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.7/467.7 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 824.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.7/224.7 kB 2.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)

In [3]:
from youtube_comment_downloader import YoutubeCommentDownloader


downloader = YoutubeCommentDownloader()


url = 'https://www.youtube.com/watch?v=r7EWCmzXw48'


comments = downloader.get_comments_from_url(url)

with open('comments.txt', 'w', encoding='utf-8') as file:
    for comment in comments:
        file.write(comment['text'] + '\n')

print(f"Saved comments to comments.txt")


Saved comments to comments.txt


In [4]:
import threading
from queue import Queue
from youtube_comment_downloader import YoutubeCommentDownloader
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer


downloader = YoutubeCommentDownloader()

model = SentenceTransformer('all-MiniLM-L6-v2')

url = 'https://www.youtube.com/watch?v=r7EWCmzXw48'

comments = downloader.get_comments_from_url(url)
comments_text = [comment['text'] for comment in comments]

batch_size = 1000
dimension = 384
index = faiss.IndexFlatIP(dimension)

def process_batch(batch_comments):

    batch_embeddings = model.encode(batch_comments)

    batch_embeddings = np.array(batch_embeddings, dtype=np.float32)


    index.add(batch_embeddings)


def worker():
    while True:
        item = q.get()
        if item is None:
            break
        process_batch(item)
        q.task_done()


q = Queue()
num_threads = 8

threads = []
for _ in range(num_threads):
    t = threading.Thread(target=worker)
    t.start()
    threads.append(t)


for i in range(0, len(comments_text), batch_size):
    batch_comments = comments_text[i:i+batch_size]
    q.put(batch_comments)


q.join()

for _ in range(num_threads):
    q.put(None)
for t in threads:
    t.join()

faiss.write_index(index, 'comments_vector_db.index')

print(f"Saved {len(comments_text)} comments to vector database 'comments_vector_db.index'")


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Saved 387 comments to vector database 'comments_vector_db.index'
